In [1]:
!pip install bayesian-optimization

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# !cp -r /content/drive/MyDrive/hack4retail/* .

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta, datetime
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
tqdm.pandas()

pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

In [5]:
# !unzip test_data.csv.zip -d .

In [6]:
# !unzip Train.zip

In [7]:
!pip install darts

In [8]:
import pandas as pd
from darts import TimeSeries
import matplotlib.pyplot as plt
from darts.metrics import mape, mase
import numpy as np
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose

from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    Prophet,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RNNModel,
    NBEATSModel,
    RegressionEnsembleModel,
    RegressionModel,
    TransformerModel,
    Theta,
    FFT
)

In [203]:
sku_final = pd.read_csv('sku_final.csv')
geo_params = pd.read_csv('geo_params.csv')
train_data = pd.read_csv('train_final.csv', parse_dates=['date'])

test_data = pd.read_csv('test_data.csv', parse_dates=['date'])
sample_submission = pd.read_csv('sample_final.csv')

In [204]:
# train_data = train_data[train_data.price.notna()]

In [205]:
train_data.sales = train_data.sales.fillna(0)

In [206]:
test_data['unqiue_id'] = test_data['geoCluster'].astype(str) + '_' + test_data['SKU'].astype(str)
train_data['unqiue_id'] = train_data['geoCluster'].astype(str) + '_' + train_data['SKU'].astype(str)

In [207]:
train_data.date.max()

Timestamp('2021-07-19 00:00:00')

In [208]:
# ml_part = train_data[train_data.date > pd.datetime(2021,7,2)].groupby(['unqiue_id']).sum()['sales'].reset_index()

In [209]:
# date_filter = train_data[train_data.date > pd.datetime(2021,7,2)]

In [16]:
# date_filter[date_filter.unqiue_id == '1894_32485']

In [17]:
# date_filter['is_zero'] = date_filter.sales == 0
# zero_filter = date_filter.groupby('unqiue_id').sum('is_zero').sort_values('is_zero', ascending=False)

In [18]:
# zero_filter = zero_filter[zero_filter['is_zero'] <= 3]

In [19]:
# zero_filter = zero_filter[zero_filter.sales > 10]

In [20]:
# zero_filter

In [21]:
# zero_filter.sales.sum()

In [22]:
# zero_filter.shape

In [23]:
# zero_filter.sort_values('sales',ascending=False).head(460).sales.plot(kind='bar')

In [24]:
# ml_part.sort_values('sales',ascending=False).head(460)

In [25]:
# ml_part.sort_values('sales',ascending=False).head(460).sales.sum()

In [26]:
# zero_filter.sales.sum()

In [27]:
# ml_part.sort_values('sales',ascending=False).head(460)

In [28]:
# ml_part = ml_part.sort_values('sales',ascending=False).head(460)

In [29]:
# ml_part = zero_filter.reset_index()

In [30]:
# ml_part[ml_part.unqiue_id == '1894_32485']

In [31]:
# train_data_not_used = train_data[~train_data.unqiue_id.isin(ml_part.unqiue_id)]

In [32]:
# train_data = train_data[train_data.unqiue_id.isin(ml_part.unqiue_id)]

In [33]:
# test_data = test_data[test_data.unqiue_id.isin(ml_part.unqiue_id)]

In [34]:
# train_data = train_data[train_data.sales != 0]

In [35]:
# # map geo-city data
# test_data = pd.merge(test_data, geo_params, on='geoCluster')
# train_data = pd.merge(train_data, geo_params, on='geoCluster')

In [36]:
# train_data[train_data['unqiue_id'] == '1894_32485']

In [37]:
# train_data[train_data['unqiue_id'] == '1894_32485'][['date', 'sales']]['sales'].describe()

In [201]:
to_improve_200 = pd.read_csv('/content/800_ids.csv')
to_improve_200['unqiue_id']

0      2043_362721
1      2233_219078
2      3209_819150
3      3209_474025
4      2158_224200
          ...     
794     2024_32485
795     2091_32485
796     2033_16649
797     2015_16649
798     2290_16649
Name: unqiue_id, Length: 799, dtype: object

In [39]:
# SKU_LIST = to_improve_200['unqiue_id']
# model =  ARIMA()
# results = dict()
# def eval_model(model, train, val):
#   model.fit(train)
#   forecast = model.predict(len(val))
#   return forecast

# val_days = 14
# for id in tqdm(SKU_LIST):
#   sku_df = train_data[train_data['unqiue_id'] == id][['date', 'sales']]
#   series = TimeSeries.from_dataframe(sku_df, 'date', 'sales')
#   train, val = series[:-val_days], series[-val_days:]
#   forecast = eval_model(model, train, val)
#   try:
#     metric = mape(val, forecast)
#     results[str(id)] = metric
#   except:
#     pass

In [202]:
test_data['pred'] = 0.0
to_improve_200['unqiue_id']

0      2043_362721
1      2233_219078
2      3209_819150
3      3209_474025
4      2158_224200
          ...     
794     2024_32485
795     2091_32485
796     2033_16649
797     2015_16649
798     2290_16649
Name: unqiue_id, Length: 799, dtype: object

In [95]:
res = test_data['unqiue_id'][to_improve_200['unqiue_id']]

KeyError: ignored

In [ ]:
SKU_LIST = to_improve_200['unqiue_id']
model =  AutoARIMA()
results_f  = dict()
def eval_model(model, train, val):
  model.fit(train)
  forecast = model.predict(val)
  return forecast

for id in tqdm(SKU_LIST):
  sku_df = train_data[train_data['unqiue_id'] == id][['date', 'sales']]
  sku_df_test = test_data[test_data['unqiue_id'] == id][['date']]
  series = TimeSeries.from_dataframe(sku_df, 'date', 'sales')
  if(len(sku_df)>30):
    forecast = eval_model(model, series, 14)
  # ids = test_data.query(f"unqiue_id == {str(id)}")["ID"]
    results_f[id] = forecast.values().ravel()
  # test_data.query(f"unqiue_id == {str(id)}")["ID"] = forecast.values().ravel()


 94%|█████████▍| 753/799 [1:11:55<03:26,  4.49s/it]

In [ ]:
ss = sample_submission
for id, forecast in tqdm((results_f.items())):
  print(id)
  df_temp = pd.DataFrame(columns = ['ID', 'pred'])
  df_temp['ID'] = test_data[test_data['unqiue_id'] == id]['ID']
  df_temp['sales'] = forecast
  for i, value in enumerate(df_temp['ID']):
    ss.loc[ss['ID'] == value, 'sales'] = forecast[i]
  # ss[ss['ID'] == test_data[test_data['unqiue_id'] == id]['ID']]['sales'] = forecast
  # ss = pd.merge(ss, df_temp[['ID', 'sales']], how='left', on=['ID'])

In [169]:
ss['sales'].describe()

0    1666028
Name: sales, dtype: int64

In [ ]:
ss

In [ ]:
ss = ss[['ID', 'sales']].fillna(0)

In [178]:
ss

,ID,sales
0,RR1666030,0.0
1,RR1666031,0.0
2,RR1666032,0.0
3,RR1666033,0.0
4,RR1666034,0.0
...,...,...
1666023,RR3332053,0.0
1666024,RR3332054,0.0
1666025,RR3332055,0.0
1666026,RR3332056,0.0


In [179]:
ss.sales.sum()

7535.966561418104

In [ ]:
ss_not_null = ss[ss['sales']!=0]

In [ ]:
ss_not_null

In [ ]:
ss.to_csv('arima_800.csv', index = False)

In [ ]:
from google.colab import files
files.download('arima_800.csv') 

In [189]:
subm = pd.read_csv('/content/submission_last.csv')

In [191]:
subm['sales'].describe()

count    1.666028e+06
mean     3.948927e-02
std      6.615413e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      7.650672e+01
Name: sales, dtype: float64

In [192]:
# subm = subm.sort(columns=['ID','sales'])
# ss_not_null = ss_not_null.sort(columns=['ID','sales'])
subm.update(ss_not_null)

In [193]:
subm['sales'].describe()

count    1.666028e+06
mean     4.401258e-02
std      6.746580e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      7.650672e+01
Name: sales, dtype: float64

In [187]:
subm.value_counts()

ID         sales
RR1666030  0.0      1
RR2776711  0.0      1
RR2776721  0.0      1
RR2776720  0.0      1
RR2776719  0.0      1
                   ..
RR2221370  0.0      1
RR2221369  0.0      1
RR2221368  0.0      1
RR2221367  0.0      1
RR3332057  0.0      1
Length: 1666028, dtype: int64

In [196]:
subm

,ID,sales
0,RR1666030,0.0
1,RR1666031,0.0
2,RR1666032,0.0
3,RR1666033,0.0
4,RR1666034,0.0
...,...,...
1666023,RR3332053,0.0
1666024,RR3332054,0.0
1666025,RR3332055,0.0
1666026,RR3332056,0.0


In [ ]:
subm

In [ ]:
subm.loc[subm['ID'] == ss_not_null['ID'], sales] = ss_not_null['']

In [195]:
subm.to_csv('arima.csv', index = False)

In [200]:
(subm['sales']>0).value_counts()

False    1652784
True       13244
Name: sales, dtype: int64

In [199]:
1652784/14

118056.0

In [139]:
results_f

{'2230_32550': array([1.55583908, 1.58559902, 1.5623313 , 1.5805231 , 1.56629989,
        1.57742026, 1.56872583, 1.57552355, 1.57020877, 1.57436411,
        1.57111527, 1.57365537, 1.5716694 , 1.57322212])}

In [ ]:
  sample_submission[sample_submission['ID'] == ids] = forecast.values().ravel()

In [135]:
len(sample_submission[sample_submission['ID'] == id])

0

In [136]:
sample_submission

,ID,sales
0,RR1666030,0
1,RR1666031,0
2,RR1666032,0
3,RR1666033,0
4,RR1666034,0
...,...,...
1666023,RR3332053,0
1666024,RR3332054,0
1666025,RR3332055,0
1666026,RR3332056,0


In [132]:
test_data[test_data['unqiue_id'] == '1894_32485']

,ID,geoCluster,SKU,date,price,unqiue_id,pred
29372,RR1695402,1894,32485,2021-07-20 00:00:00,36.89,1894_32485,0.0
29373,RR1695403,1894,32485,2021-07-21 00:00:00,36.89,1894_32485,0.0
29374,RR1695404,1894,32485,2021-07-22 00:00:00,36.89,1894_32485,0.0
29375,RR1695405,1894,32485,2021-07-23 00:00:00,34.89,1894_32485,0.0
29376,RR1695406,1894,32485,2021-07-24 00:00:00,34.89,1894_32485,0.0
29377,RR1695407,1894,32485,2021-07-25 00:00:00,34.89,1894_32485,0.0
29378,RR1695408,1894,32485,2021-07-26 00:00:00,34.89,1894_32485,0.0
29379,RR1695409,1894,32485,2021-07-27 00:00:00,34.89,1894_32485,0.0
29380,RR1695410,1894,32485,2021-07-28 00:00:00,34.89,1894_32485,0.0
29381,RR1695411,1894,32485,2021-07-29 00:00:00,34.89,1894_32485,0.0


In [114]:
id

'2230_32550'

In [124]:
test_data.query(f"unqiue_id == {id}")["pred"] = forecast.values().ravel()

In [113]:
print(len(test_data.loc[test_data['unqiue_id'] == id]))

0


In [107]:
print(sample_submission.group_by['id'])

AttributeError: ignored

In [129]:
res.value_counts()

Series([], dtype: int64)

In [130]:
test_data['pred'].value_counts()

0.0    1666028
Name: pred, dtype: int64

In [77]:
results

{'2230_32550': 112.69111394607604,
 '2177_32550': 147.30730852658988,
 '1991_32550': 189.74745546391827,
 '2966_32550': 388.8731427299889,
 '3209_144184': 162.34165457964156,
 '2042_26194': 69.7245626338212}

In [ ]:
# extract seasonality
for df in [train_data, test_data]:
  df['quarter'] = df['date'].dt.quarter
  df['month'] = df['date'].dt.month
  df['dayofweek'] = df['date'].dt.dayofweek
  df['week'] = df['date'].dt.week
  df['season'] = df['month'].apply(lambda x: (x%12 + 3)//3)


In [ ]:
# fix price
median_price = train_data[train_data.price.notna()].groupby('unqiue_id').median()['price'].reset_index().rename(columns={'price':'median_price'})
train_data = pd.merge(train_data, median_price, how='left', on=['unqiue_id'])
train_data['price'] = train_data['price'].fillna(train_data['median_price'])

In [ ]:
# difference between price and median_price

In [ ]:
test_data = pd.merge(test_data, median_price, how='left', on=['unqiue_id'])

train_data['price_diff'] = train_data['price'] - train_data['median_price']
test_data['price_diff'] = test_data['price'] - test_data['median_price']

train_data = train_data.drop(columns=['median_price'])
test_data = test_data.drop(columns=['median_price'])

In [ ]:
# train_data.groupby('unqiue_id').tail(14).groupby('unqiue_id').sum().sort_values('sales').head(50)

In [ ]:
#sku info

In [ ]:
sku_final = sku_final[['SKU', 'productCategoryId','productCategory_caption_ENG', 'productTypeId', 'productType_caption_ENG', 'brandId', 'lagerUnitQuantity',
           'lagerUnitTypeId', 'lagerUnitType_caption', 'trademark', 'countryOfOrigin', 'countryOfOrigin_caption',
           'commodity_group', 'commodity_group_caption_ENG']]

In [ ]:
# easy_features = sku_final[['SKU', 'productCategoryId', 'productTypeId', 'brandId', 'lagerUnitQuantity', 'lagerUnitTypeId',
#            'trademark', 'countryOfOrigin', 'commodity_group']]

easy_features = sku_final[['SKU', 'productCategoryId', 'commodity_group']]

In [ ]:
easy_features.isna().sum() / easy_features.shape[0]

SKU                  0.0
productCategoryId    0.0
commodity_group      0.0
dtype: float64

In [ ]:
train_data = pd.merge(train_data, easy_features, how='left', on=['SKU'])
test_data = pd.merge(test_data, easy_features, how='left', on=['SKU'])

In [ ]:
train_data.head()

,ID,geoCluster,SKU,date,price,sales,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,productCategoryId,commodity_group
0,RR27972816,1894,32485,2020-11-18,35.49,1.0,1894_32485,0,4,11,2,47,4,-5.6,5381.0,5551018
1,RR27972817,1894,32485,2020-11-19,35.49,1.4,1894_32485,0,4,11,3,47,4,-5.6,5381.0,5551018
2,RR27972963,1894,32485,2021-04-14,46.79,6.5,1894_32485,0,2,4,2,15,2,5.7,5381.0,5551018
3,RR27972965,1894,32485,2021-04-16,45.39,5.3,1894_32485,0,2,4,4,15,2,4.3,5381.0,5551018
4,RR27972966,1894,32485,2021-04-17,43.99,2.0,1894_32485,0,2,4,5,15,2,2.9,5381.0,5551018


In [ ]:
# combo features

In [ ]:
train_data['is_train'] = 1
test_data['sales'] = np.NaN
test_data['is_train'] = 0

In [ ]:
combo_data = pd.concat([train_data, test_data])

In [ ]:
combo_data

,ID,geoCluster,SKU,date,price,sales,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,productCategoryId,commodity_group,is_train
0,RR27972816,1894,32485,2020-11-18,35.49,1.0,1894_32485,0,4,11,2,47,4,-5.6,5381.0,5551018,1
1,RR27972817,1894,32485,2020-11-19,35.49,1.4,1894_32485,0,4,11,3,47,4,-5.6,5381.0,5551018,1
2,RR27972963,1894,32485,2021-04-14,46.79,6.5,1894_32485,0,2,4,2,15,2,5.7,5381.0,5551018,1
3,RR27972965,1894,32485,2021-04-16,45.39,5.3,1894_32485,0,2,4,4,15,2,4.3,5381.0,5551018,1
4,RR27972966,1894,32485,2021-04-17,43.99,2.0,1894_32485,0,2,4,5,15,2,2.9,5381.0,5551018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,RR3331283,3230,838137,2021-07-29,49.79,NaN,3230_838137,0,3,7,3,30,3,10.0,5422.0,5551018,0
11784,RR3331284,3230,838137,2021-07-30,49.79,NaN,3230_838137,0,3,7,4,30,3,10.0,5422.0,5551018,0
11785,RR3331285,3230,838137,2021-07-31,49.79,NaN,3230_838137,0,3,7,5,30,3,10.0,5422.0,5551018,0
11786,RR3331286,3230,838137,2021-08-01,49.79,NaN,3230_838137,0,3,8,6,30,3,10.0,5422.0,5551018,0


In [ ]:
# lags = [14, 15, 16, 17, 18, 19, 20, 21]
# windows = [14, 21]

# for lag in lags:
#     combo_data[f'lag_{lag}'] = combo_data.groupby(['unqiue_id'])['sales'].shift(lag)
#     # combo_data[f'exp_median_{lag}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.expanding().median())
    
#     for window in windows:
        
#         combo_data[f'rmedian_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).median())
#         # combo_data[f'rstd_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).std())
#         # combo_data[f'cv_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).apply(cv))


In [ ]:
def cv(x):
    mean_data = np.mean(x)

    if mean_data == 0:
        return np.nan
    else:
        stddev = np.std(x)

        return stddev / mean_data

In [ ]:
lags = [14, 15, 16, 17, 18, 19, 20, 21]
windows = [14, 21]

for lag in lags:
    combo_data[f'lag_{lag}'] = combo_data.groupby(['unqiue_id'])['sales'].shift(lag)
    combo_data[f'exp_median_{lag}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.expanding().median())
    
    for window in windows:
        
        combo_data[f'rmedian_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).median())
        combo_data[f'rstd_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).std())
        # combo_data[f'cv_{lag}_{window}'] = combo_data.groupby(['unqiue_id'])[f'lag_{lag}'].transform(lambda x: x.rolling(window).apply(cv))

In [ ]:
train_after_combo = combo_data[combo_data.is_train == 1]
test_after_combo = combo_data[combo_data.is_train == 0]

In [ ]:
# train_after_combo = train_after_combo.dropna()

In [ ]:
X_train = train_after_combo.drop(columns=['ID', 'is_train']).reset_index(drop=True)
y_train = train_after_combo['sales'].reset_index(drop=True)

X_test = test_after_combo.drop(columns=['ID', 'sales', 'date', 'is_train']).reset_index(drop=True)

In [ ]:
le = LabelEncoder()
X_train['unqiue_id'] = le.fit_transform(X_train['unqiue_id'])

le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
X_test['unqiue_id'] = X_test['unqiue_id'].apply(lambda x: le_dict.get(x, -99999999))

In [ ]:
y_train

0         1.0
1         1.4
2         6.5
3         5.3
4         2.0
         ... 
171428    6.0
171429    6.0
171430    3.0
171431    3.0
171432    6.0
Name: sales, Length: 171433, dtype: float64

In [ ]:
X_train.columns

Index(['geoCluster', 'SKU', 'date', 'price', 'sales', 'unqiue_id', 'cityId',
       'quarter', 'month', 'dayofweek', 'week', 'season', 'price_diff',
       'productCategoryId', 'commodity_group', 'lag_14', 'exp_median_14',
       'rmedian_14_14', 'rstd_14_14', 'rmedian_14_21', 'rstd_14_21', 'lag_15',
       'exp_median_15', 'rmedian_15_14', 'rstd_15_14', 'rmedian_15_21',
       'rstd_15_21', 'lag_16', 'exp_median_16', 'rmedian_16_14', 'rstd_16_14',
       'rmedian_16_21', 'rstd_16_21', 'lag_17', 'exp_median_17',
       'rmedian_17_14', 'rstd_17_14', 'rmedian_17_21', 'rstd_17_21', 'lag_18',
       'exp_median_18', 'rmedian_18_14', 'rstd_18_14', 'rmedian_18_21',
       'rstd_18_21', 'lag_19', 'exp_median_19', 'rmedian_19_14', 'rstd_19_14',
       'rmedian_19_21', 'rstd_19_21', 'lag_20', 'exp_median_20',
       'rmedian_20_14', 'rstd_20_14', 'rmedian_20_21', 'rstd_20_21', 'lag_21',
       'exp_median_21', 'rmedian_21_14', 'rstd_21_14', 'rmedian_21_21',
       'rstd_21_21'],
      dtype='o

In [ ]:
X_train = X_train[['geoCluster', 'SKU', 'price', 'unqiue_id', 'cityId',
       'quarter', 'month', 'dayofweek', 'week', 'season', 'price_diff',
       'lag_14', 'rmedian_14_14', 'rmedian_14_21', 'lag_15', 'rmedian_15_14',
       'rmedian_15_21', 'lag_16', 'rmedian_16_14', 'rmedian_16_21', 'lag_17',
       'rmedian_17_14', 'rmedian_17_21', 'lag_18', 'rmedian_18_14',
       'rmedian_18_21', 'lag_19', 'rmedian_19_14', 'rmedian_19_21', 'lag_20',
       'rmedian_20_14', 'rmedian_20_21', 'lag_21', 'rmedian_21_14',
       'rmedian_21_21', 'date', 'sales']]

In [ ]:
X_test = X_test[['geoCluster', 'SKU', 'price', 'unqiue_id', 'cityId',
       'quarter', 'month', 'dayofweek', 'week', 'season', 'price_diff',
       'lag_14', 'rmedian_14_14', 'rmedian_14_21', 'lag_15', 'rmedian_15_14',
       'rmedian_15_21', 'lag_16', 'rmedian_16_14', 'rmedian_16_21', 'lag_17',
       'rmedian_17_14', 'rmedian_17_21', 'lag_18', 'rmedian_18_14',
       'rmedian_18_21', 'lag_19', 'rmedian_19_14', 'rmedian_19_21', 'lag_20',
       'rmedian_20_14', 'rmedian_20_21', 'lag_21', 'rmedian_21_14',
       'rmedian_21_21']]

In [ ]:
X_train

,geoCluster,SKU,price,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,lag_14,rmedian_14_14,rmedian_14_21,lag_15,rmedian_15_14,rmedian_15_21,lag_16,rmedian_16_14,rmedian_16_21,lag_17,rmedian_17_14,rmedian_17_21,lag_18,rmedian_18_14,rmedian_18_21,lag_19,rmedian_19_14,rmedian_19_21,lag_20,rmedian_20_14,rmedian_20_21,lag_21,rmedian_21_14,rmedian_21_21,date,sales
0,1894,32485,35.49,0,0,4,11,2,47,4,-5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-18,1.0
1,1894,32485,35.49,0,0,4,11,3,47,4,-5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-19,1.4
2,1894,32485,46.79,0,0,2,4,2,15,2,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-14,6.5
3,1894,32485,45.39,0,0,2,4,4,15,2,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-16,5.3
4,1894,32485,43.99,0,0,2,4,5,15,2,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-17,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171428,2904,781766,113.59,719,0,3,7,2,28,3,0.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2.0,3.0,3.0,4.0,3.0,3.0,2021-07-14,6.0
171429,2904,781766,113.59,719,0,3,7,3,28,3,0.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2.0,3.0,3.0,2021-07-15,6.0
171430,2904,781766,113.59,719,0,3,7,4,28,3,0.0,2.0,3.0,2.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2021-07-16,3.0
171431,2904,781766,113.59,719,0,3,7,5,28,3,0.0,8.0,3.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,2021-07-17,3.0


In [ ]:
X_test

,geoCluster,SKU,price,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,lag_14,rmedian_14_14,rmedian_14_21,lag_15,rmedian_15_14,rmedian_15_21,lag_16,rmedian_16_14,rmedian_16_21,lag_17,rmedian_17_14,rmedian_17_21,lag_18,rmedian_18_14,rmedian_18_21,lag_19,rmedian_19_14,rmedian_19_21,lag_20,rmedian_20_14,rmedian_20_21,lag_21,rmedian_21_14,rmedian_21_21
0,491,674893,107.89,838,1,3,7,1,29,3,0.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0,3.0,3.5,4.0
1,491,674893,107.89,838,1,3,7,2,29,3,0.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0
2,491,674893,107.89,838,1,3,7,3,29,3,0.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0
3,491,674893,107.89,838,1,3,7,4,29,3,0.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0
4,491,674893,107.89,838,1,3,7,5,29,3,0.0,12.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,3230,838137,49.79,837,0,3,7,3,30,3,10.0,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN,12.0,NaN,NaN
11784,3230,838137,49.79,837,0,3,7,4,30,3,10.0,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN
11785,3230,838137,49.79,837,0,3,7,5,30,3,10.0,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN
11786,3230,838137,49.79,837,0,3,8,6,30,3,10.0,39.0,15.0,NaN,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
cat_features = ['geoCluster', 'SKU', 'unqiue_id', 'cityId', 'quarter', 'month', 'dayofweek', 'week', 'season']

In [ ]:
categorical_features = [X_train.columns.get_loc(i) for i in cat_features]

In [ ]:
categorical_features

[0, 1, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
train_data_not_used['pred'] = 0
train_data_not_used['diff'] = np.abs(train_data_not_used['pred'] - train_data_not_used['sales'])

In [ ]:
train_data_not_used

,ID,geoCluster,SKU,date,price,sales,unqiue_id,pred,diff
0,RR27956447,21,32485,2021-07-08,39.69,0.9,21_32485,0,0.9
1,RR27956448,21,32485,2021-07-09,NaN,0.0,21_32485,0,0.0
2,RR27956449,21,32485,2021-07-10,NaN,0.0,21_32485,0,0.0
3,RR27956450,21,32485,2021-07-11,NaN,0.0,21_32485,0,0.0
4,RR27956451,21,32485,2021-07-12,NaN,0.0,21_32485,0,0.0
...,...,...,...,...,...,...,...,...,...
27956440,RR55912887,3230,841486,2021-07-15,NaN,0.0,3230_841486,0,0.0
27956441,RR55912888,3230,841486,2021-07-16,NaN,0.0,3230_841486,0,0.0
27956442,RR55912889,3230,841486,2021-07-17,NaN,0.0,3230_841486,0,0.0
27956443,RR55912890,3230,841486,2021-07-18,NaN,0.0,3230_841486,0,0.0


In [ ]:
train_data_not_used[train_data_not_used.date == pd.datetime(2021,7,5)]['sales'].sum()

15250.25

In [ ]:
# X_train = X_train[['geoCluster', 'SKU', 'date', 'sales', 'unqiue_id', 'cityId']]

In [ ]:
# X_test = X_test[['geoCluster', 'SKU', 'unqiue_id', 'cityId']]

In [ ]:
END_TRAIN   = 1913
P_HORIZON   = 14

In [ ]:
# def accuracy_feval(y_pred, y_test):
    
#     y_pred = np.array(y_pred) 
#     y_pred[y_pred < 0 ] = 0
    
#     y_test = y_test.get_label()
#     y_test = np.array(y_test)
    
#     n_step = 5
#     err = 0
#     for i in range(n_step, len(y_test), n_step):
#         err += np.abs(y_test[i-n_step:i].sum() - y_pred[i-n_step:i].sum())
    
#     accuracy = (1 - (err / y_test.sum())) * 100
    
#     return 'accuracy', accuracy, True

def mse_feval(y_pred, y_test):
    
    y_pred = np.array(y_pred) 
    y_pred[y_pred < 0 ] = 0
    
    y_test = y_test.get_label()
    y_test = np.array(y_test)

    res = mean_squared_error(y_test, y_pred)
    
    # n_step = 5
    # err = 0
    # for i in range(n_step, len(y_test), n_step):
    #     err += np.abs(y_test[i-n_step:i].sum() - y_pred[i-n_step:i].sum())
    
    # accuracy = (1 - (err / y_test.sum())) * 100
    
    return 'mse', res, False


best_iterations = None
max_score = -np.inf
bestmodel = None
bestmodel_local = np.inf

def bayes_parameter_opt_lgb(X_train_passed, y_train_passed, init_round=200, opt_round=250, random_seed=6, output_process=False):

    # cv = TimeSeriesSplit(n_splits=5)

    def lgb_eval(
        num_iterations,
                 learning_rate,
                 num_leaves,
                 min_child_samples):
        
        params = {'application':'regression',
                  # 'early_stopping_round':10,
                  # 'metric':'custom',
                  # 'objective':'tweedie',
                  'categorical_feature':categorical_features,
                 }
        params["num_leaves"] = int(round(num_leaves))
        params["learning_rate"] = max(min(learning_rate, 1), 0)
        params["num_iterations"] = int(round(num_iterations))
        params['min_child_samples'] = int(min_child_samples)
        params['verbose'] = -1
        
        scores = []
        curr_iterations = []

        global best_iterations
        global max_score
        global bestmodel
        global bestmodel_local
        
        n_folds = 5
        test_period = 14
        for fold in range(1, n_folds+1):
            print(f'validation: {fold}')
            X_train_fold = X_train_passed[X_train_passed.date < X_train_passed.date.max() - timedelta(days=test_period*fold)]
            X_val_fold = X_train_passed[(X_train_passed.date > X_train_fold.date.max()) & (X_train_passed.date <= X_train_fold.date.max() + timedelta(days=test_period))]

            y_train_fold, y_val_fold = y_train_passed[X_train_fold.index], y_train_passed[X_val_fold.index]

            static_part = train_data_not_used[train_data_not_used.date.isin(X_val_fold.date)]
            print(X_train_fold.date.min(), X_train_fold.date.max())
            print(X_val_fold.date.min(), X_val_fold.date.max())
            # print(static_part.date.min(), static_part.date.max())

            X_val_date = X_val_fold.date
            X_val_sales = X_val_fold.sales
            X_train_fold = X_train_fold.drop(columns=['date', 'sales'])
            X_val_fold = X_val_fold.drop(columns=['date', 'sales'])

 
            train_data = lgb.Dataset(data=X_train_fold, label=y_train_fold,
                            free_raw_data = False)
            
            test_data = lgb.Dataset(data=X_val_fold, label=y_val_fold,
                                        free_raw_data = False)

            model = lgb.train(params, train_data, valid_sets = [test_data], verbose_eval = None)
            
            # print(model.params)

            curr_pred = model.predict(X_val_fold)
            curr_pred[curr_pred < 0 ] = 0
            
            
            X_val_fold['pred'] = curr_pred
            X_val_fold['date'] = X_val_date
            X_val_fold['sales'] = X_val_sales
            X_val_fold['diff'] = np.abs(X_val_fold['pred'] - X_val_fold['sales'])
            # print()
            # print(X_val_fold['diff'].sum())
            # print(X_val_fold['sales'].sum())
            # print(X_val_fold['pred'].sum())
            # print(X_val_fold['unqiue_id'].nunique())

            # print(X_val_fold[X_val_fold.date == pd.datetime(2021,7,5)].sales.sum())
            X_val_fold = X_val_fold[(X_val_fold.date <= X_val_fold.date.max() - timedelta(days=7))]
            print('first 7 days:')
            print(X_val_fold.date.min(), X_val_fold.date.max())
            ml_part = X_val_fold[['diff', 'sales', 'date']]
            static_part = static_part[['diff', 'sales', 'date']]

            com_res = pd.concat([ml_part,static_part])

            com_res = com_res.groupby('date').sum()[['diff', 'sales']]
            com_res['mae_precentage'] = com_res['diff'] / com_res['sales']
            # print(f'com_res: {com_res}')
            score = com_res['mae_precentage'].mean()

            if fold == 0 and score < bestmodel_local:
              print('here')
              bestmodel = model
              bestmodel_local = score
            
            # print(model.best_score)
            scores.append(-1 * score)
            curr_iterations.append(model.best_iteration)
            
            print(f'score: {score}')
            print(model.best_iteration)
        if max_score < np.mean(scores):
            best_iterations = curr_iterations.copy()
            max_score = np.mean(scores)
        return np.mean(scores)


    
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.0003, 0.35),
                                            'num_iterations': (10, 45),
                                            'num_leaves': (3, 255),
                                            'min_child_samples': (98, 555)
                                           }, random_state=42)
        
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    
    return lgbBO.res


In [ ]:
INIT_STEPS = 1
OPTIMIZATION_STEPS = 1
opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=INIT_STEPS, opt_round=OPTIMIZATION_STEPS)

|   iter    |  target   | learni... | min_ch... | num_it... | num_le... |
-------------------------------------------------------------------------
validation: 1
2020-04-02 00:00:00 2021-07-04 00:00:00
2021-07-05 00:00:00 2021-07-18 00:00:00
first 7 days:
2021-07-05 00:00:00 2021-07-11 00:00:00
score: 0.9381834814591167
0
validation: 2
2020-04-02 00:00:00 2021-06-20 00:00:00
2021-06-21 00:00:00 2021-07-04 00:00:00
first 7 days:
2021-06-21 00:00:00 2021-06-27 00:00:00
score: 0.9443962655804705
0
validation: 3
2020-04-02 00:00:00 2021-06-06 00:00:00
2021-06-07 00:00:00 2021-06-20 00:00:00
first 7 days:
2021-06-07 00:00:00 2021-06-13 00:00:00
score: 0.9450476609070018
0
validation: 4
2020-04-02 00:00:00 2021-05-23 00:00:00
2021-05-24 00:00:00 2021-06-06 00:00:00
first 7 days:
2021-05-24 00:00:00 2021-05-30 00:00:00
score: 0.9453301923081173
0
validation: 5
2020-04-02 00:00:00 2021-05-09 00:00:00
2021-05-10 00:00:00 2021-05-23 00:00:00
first 7 days:
2021-05-10 00:00:00 2021-05-16 00:00:00


In [ ]:
bestmodel

In [ ]:
best_params = sorted(opt_params, key=lambda k: k['target'], reverse=True)[0]['params']

In [ ]:
opt_params

[{'params': {'learning_rate': 0.13127667956092265,
   'min_child_samples': 532.4764380293317,
   'num_iterations': 35.619787963399176,
   'num_leaves': 153.86193801765322},
  'target': -0.9435007224062492},
 {'params': {'learning_rate': 0.2531764735823216,
   'min_child_samples': 403.3035528494258,
   'num_iterations': 17.799223222241046,
   'num_leaves': 19.101627847034514},
  'target': -0.9435801482609547}]

In [ ]:
best_params

{'learning_rate': 0.13127667956092265,
 'min_child_samples': 532.4764380293317,
 'num_iterations': 35.619787963399176,
 'num_leaves': 153.86193801765322}

In [ ]:
# params = {
#      'learning_rate': max(min(best_params['learning_rate'], 1), 0),
#      'min_child_samples': int(best_params['min_child_samples']),
#      'num_iterations': int(np.mean(best_iterations)),
#      'num_leaves': int(round(best_params['num_leaves'])),
#      'application':'regression',
#      'categorical_feature':categorical_features}

# params = {
#      'learning_rate': 0.0013,
#      'min_child_samples': 532,
#      'num_iterations': 152,
#      'num_leaves': 154,
#      'application':'regression',
#     #  'categorical_feature':categorical_features,
#      }
    #  {'application': 'regression', 'metric': 'custom', 'num_leaves': 154, 'learning_rate': 0.13127667956092265, 'min_child_samples': 532, 'verbose': -1}

# params = {'application': 'regression',
#  'categorical_feature': [0, 1, 5, 6, 7, 8, 9, 10, 11],
#  'learning_rate': 0.054859718562720046,
#  'min_child_samples': 169,
#  'num_iterations': 87,
#  'num_leaves': 221
#  'categorical_feature':categorical_features}

params = {'application': 'regression',
    'learning_rate': 0.13127667956092265,
 'min_child_samples': 532,
 'num_iterations': 35,
 'num_leaves': 154}

#  {'application': 'regression', 'num_leaves': 154, 'learning_rate': 0.13127667956092265, 'min_child_samples': 532, 'verbose': -1}

In [ ]:
params

{'application': 'regression',
 'learning_rate': 0.13127667956092265,
 'min_child_samples': 532,
 'num_iterations': 35,
 'num_leaves': 154}

In [ ]:
X_test.shape

(11788, 35)

In [ ]:
X_test

,geoCluster,SKU,price,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,lag_14,rmedian_14_14,rmedian_14_21,lag_15,rmedian_15_14,rmedian_15_21,lag_16,rmedian_16_14,rmedian_16_21,lag_17,rmedian_17_14,rmedian_17_21,lag_18,rmedian_18_14,rmedian_18_21,lag_19,rmedian_19_14,rmedian_19_21,lag_20,rmedian_20_14,rmedian_20_21,lag_21,rmedian_21_14,rmedian_21_21
0,491,674893,107.89,838,1,3,7,1,29,3,0.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0,3.0,3.5,4.0
1,491,674893,107.89,838,1,3,7,2,29,3,0.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0
2,491,674893,107.89,838,1,3,7,3,29,3,0.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0
3,491,674893,107.89,838,1,3,7,4,29,3,0.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0
4,491,674893,107.89,838,1,3,7,5,29,3,0.0,12.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,3230,838137,49.79,837,0,3,7,3,30,3,10.0,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN,12.0,NaN,NaN
11784,3230,838137,49.79,837,0,3,7,4,30,3,10.0,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN
11785,3230,838137,49.79,837,0,3,7,5,30,3,10.0,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN
11786,3230,838137,49.79,837,0,3,8,6,30,3,10.0,39.0,15.0,NaN,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN


In [ ]:
X_train

,geoCluster,SKU,price,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,lag_14,rmedian_14_14,rmedian_14_21,lag_15,rmedian_15_14,rmedian_15_21,lag_16,rmedian_16_14,rmedian_16_21,lag_17,rmedian_17_14,rmedian_17_21,lag_18,rmedian_18_14,rmedian_18_21,lag_19,rmedian_19_14,rmedian_19_21,lag_20,rmedian_20_14,rmedian_20_21,lag_21,rmedian_21_14,rmedian_21_21,date,sales
0,1894,32485,35.49,0,0,4,11,2,47,4,-5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-18,1.0
1,1894,32485,35.49,0,0,4,11,3,47,4,-5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-19,1.4
2,1894,32485,46.79,0,0,2,4,2,15,2,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-14,6.5
3,1894,32485,45.39,0,0,2,4,4,15,2,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-16,5.3
4,1894,32485,43.99,0,0,2,4,5,15,2,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-17,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171428,2904,781766,113.59,719,0,3,7,2,28,3,0.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2.0,3.0,3.0,4.0,3.0,3.0,2021-07-14,6.0
171429,2904,781766,113.59,719,0,3,7,3,28,3,0.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2.0,3.0,3.0,2021-07-15,6.0
171430,2904,781766,113.59,719,0,3,7,4,28,3,0.0,2.0,3.0,2.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,1.0,2.5,3.0,2021-07-16,3.0
171431,2904,781766,113.59,719,0,3,7,5,28,3,0.0,8.0,3.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,1.0,2.0,3.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0,2021-07-17,3.0


In [ ]:
X_train = X_train.drop(columns=['date', 'sales'])

In [ ]:
train_data = lgb.Dataset(data=X_train, label=y_train,
                            free_raw_data = False)

model = lgb.train(params, train_data)

In [ ]:
X_test.lag_14.sum()

76362.231

In [ ]:
X_test

,geoCluster,SKU,price,unqiue_id,cityId,quarter,month,dayofweek,week,season,price_diff,lag_14,rmedian_14_14,rmedian_14_21,lag_15,rmedian_15_14,rmedian_15_21,lag_16,rmedian_16_14,rmedian_16_21,lag_17,rmedian_17_14,rmedian_17_21,lag_18,rmedian_18_14,rmedian_18_21,lag_19,rmedian_19_14,rmedian_19_21,lag_20,rmedian_20_14,rmedian_20_21,lag_21,rmedian_21_14,rmedian_21_21
0,491,674893,107.89,838,1,3,7,1,29,3,0.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0,3.0,3.5,4.0
1,491,674893,107.89,838,1,3,7,2,29,3,0.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,1.0,3.5,4.0
2,491,674893,107.89,838,1,3,7,3,29,3,0.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0
3,491,674893,107.89,838,1,3,7,4,29,3,0.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0
4,491,674893,107.89,838,1,3,7,5,29,3,0.0,12.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.5,4.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,3230,838137,49.79,837,0,3,7,3,30,3,10.0,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN,12.0,NaN,NaN
11784,3230,838137,49.79,837,0,3,7,4,30,3,10.0,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN,18.0,NaN,NaN
11785,3230,838137,49.79,837,0,3,7,5,30,3,10.0,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN,NaN
11786,3230,838137,49.79,837,0,3,8,6,30,3,10.0,39.0,15.0,NaN,5.0,15.0,NaN,24.0,15.0,NaN,19.0,12.0,NaN,2.0,11.0,NaN,12.0,NaN,NaN,23.0,NaN,NaN,18.0,NaN,NaN


In [ ]:
X_test = X_test.drop(columns=['pred', 'diff'])

KeyError: ignored

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[y_pred < 0 ] = 0

In [ ]:
y_pred

In [ ]:
sum(y_pred)

In [ ]:
X_test['pred'] = y_pred

In [ ]:
X_test['diff'] = np.abs(X_test['lag_14'] - X_test['pred'])

In [ ]:
X_test[X_test.unqiue_id == 39][['pred', 'lag_14', 'diff']]

In [ ]:
X_test[['unqiue_id', 'pred', 'lag_14', 'diff']].groupby('unqiue_id').sum().sort_values('diff', ascending=False).head(50)

In [ ]:
test_data['pred'] = y_pred

In [ ]:
test_data

In [ ]:
ss = pd.merge(sample_submission, test_data[['ID', 'pred']], how='left', on=['ID'])

In [ ]:
ss = ss[['ID', 'pred']].fillna(0).rename(columns={'pred':'sales'})

In [ ]:
ss

In [ ]:
ss.sales.sum()

In [ ]:
ss.to_csv('last_week_new_filter_88.csv', index = False)

In [ ]:
from google.colab import files
files.download('last_week_new_filter_88.csv') 